In [1]:

import pandas as pd
import re
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time


In [2]:
start_time = time.time()

data0 = pd.read_csv('C:/.../Swiggy-Zomato-Food-Aggregators-Analysis/Articles_Data/Zomato_Articles.csv')
data0 = data0.dropna()
data00 = pd.read_csv('C:/.../Swiggy-Zomato-Food-Aggregators-Analysis/Articles_Data/Swiggy_Articles.csv') 
data00 = data00.dropna()

In [3]:
'''
Impact Analysis for Zomato delivery boy incident[happened on 11 Dec 2018] and Swiggy Anonymous Blog incident 
[happened on 26 July 2017] on the two brands. Here we will look into the word frequencies for both the food aggregators.
'''

#Zomato (get before incidence, during incidence and after incidence data) -
Zomato_data_during_incidence = data0[data0.Article_date.str.contains('11 December 2018',case=False)]
Zomato_data_after_incidence = data0.loc[:(Zomato_data_during_incidence.index.values[0]-1)]
Zomato_data_before_incidence = data0.loc[(Zomato_data_during_incidence.index.values[0]+1):]

#Swiggy (get before incidence, during incidence and after incidence data) -
Swiggy_data_during_incidence = data00[data00.Article_date.str.contains('26 July 2017',case=False)]
Swiggy_data_after_incidence = data00.loc[:(Swiggy_data_during_incidence.index.values[0]-1)]
Swiggy_data_before_incidence = data00.loc[(Swiggy_data_during_incidence.index.values[0]+1):]

def wordfreq(data):
    reviews = data['Article_content']
    #reviews0 = list(reviews)
    
    '''Remove Emails ''' 
    reviews1 = [re.sub('\S*@\S*\s?', '', sent) for sent in reviews] # Remove emails
    reviews1 = [re.sub('\s+', ' ', sent) for sent in reviews1] # Remove newline character
    reviews1 = [re.sub("\'", "", sent) for sent in reviews1] # Remove distracting single quotes
    reviews2 = [re.sub('[^A-Za-z0-9]+', ' ', sent) for sent in reviews1] # remove alphanumeric values
    
    '''Remove numbers ''' 
    
    reviews3 = [''.join(i for i in s if not i.isdigit()) for s in reviews2]
    #del reviews1, reviews2
    
    '''Bigram using CountVectorizer ''' 
    
    vectorizer = CountVectorizer(min_df = 1, ngram_range=(2, 2), stop_words = 'english')
    X = vectorizer.fit_transform(reviews3)
    df = pd.DataFrame(X.toarray().transpose(), index = vectorizer.get_feature_names())
    
    frequency = list(df.sum(axis=1))
    bigramwords = list(vectorizer.get_feature_names())
    
    bigram_termfreq= pd.DataFrame({'bigramwords': bigramwords, 'frequency': frequency})
    bigram = bigram_termfreq.sort_values(['frequency'], ascending=[False]) #get bigram words and their frequency in document
     
       #min_df = 2, max_df = 0.70 for before and after incidence. min_df= 1 during incidence.
    tf = TfidfVectorizer(min_df = 1, ngram_range=(2, 2), smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word', stop_words = 'english')
    txt_fitted = tf.fit(reviews3)
    txt_transformed = tf.transform(reviews3)
    doc_term_matrix = txt_transformed.todense()
    df = pd.DataFrame(doc_term_matrix, 
                      columns= tf.get_feature_names())
    
    idf = tf.idf_
    rr = dict(zip(txt_fitted.get_feature_names(), idf))
    token_weight = pd.DataFrame.from_dict(rr, orient='index').reset_index()
    token_weight.columns=('word_token','weight')
    token_weight = token_weight.sort_values(by='weight', ascending=False)
    
    return bigram, token_weight
    
    
loop = (Zomato_data_during_incidence.dropna(), Zomato_data_after_incidence.dropna(), Zomato_data_before_incidence.dropna(), 
Swiggy_data_during_incidence.dropna(), Swiggy_data_after_incidence.dropna(), Swiggy_data_before_incidence.dropna())
for i in range(0,len(loop)):
    j=("zoma_bigram_during_incdnt", "zoma_bigram_after_incdnt", "zoma_bigram_before_incdnt",
       "swgy_bigram_during_incdnt", "swgy_bigram_after_incdnt", "swgy_bigram_before_incdnt")
    print(i)
    print(j)
    results = wordfreq(loop[i])
    bigram_termfreq = results[0] # bigram term frequency, first output of the function
    bigram_inversetermfreq = results[1] # bigram inverse term frequency, second output of the function
    #if i<3:
    with pd.ExcelWriter("C:/.../Swiggy-Zomato-Food-Aggregators-Analysis/articles_data/bigrams_output" + j[i] + ".xlsx") as writer:
        bigram_termfreq.to_excel(writer, sheet_name= 'TermFrequency')  
        bigram_inversetermfreq.to_excel(writer, sheet_name= 'tf-idf_weight')
            
            
print("--- %s seconds ---" % (time.time() - start_time))

0
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_bigram_after_incdnt', 'swgy_bigram_before_incdnt')
1
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_bigram_after_incdnt', 'swgy_bigram_before_incdnt')
2
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_bigram_after_incdnt', 'swgy_bigram_before_incdnt')
3
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_bigram_after_incdnt', 'swgy_bigram_before_incdnt')
4
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_bigram_after_incdnt', 'swgy_bigram_before_incdnt')
5
('zoma_bigram_during_incdnt', 'zoma_bigram_after_incdnt', 'zoma_bigram_before_incdnt', 'swgy_bigram_during_incdnt', 'swgy_b